##AWS Cloud Data Lake demo Overview

Components and steps we will follow in today's demo:
<br />
<br />
1. AWS Glue as our Central Metastore
2. We will launch 1 Kinesis Stream ie. **User click stream**
3. Join an already existing user Profile Delta table registered in our Glue metastore 
4. We will execute a crawler job to pull in an S3 datasets into our AWS Glue metastore
5. The pipeline consists of a Data Lake medallion appproach
6. We will demonstrate the Full DML support of Delta Lake while curating the Data Lake.  
6. The curated GOLD dataset will be available to Athena and pushed to Redshift for later consumption
7. Finally, a QuickSight dashboard

<img src="https://drive.google.com/uc?id=1fZL3IhbB86bh5c6z54TvwytjEBTNa7Wo&authuser=recohut.data.001%40gmail.com&usp=drive_fs" height="800" width="1000"/>

## Clean Up and Ingest

In [0]:
%sql 
CREATE DATABASE IF NOT EXISTS sparsha_aws_cdl
LOCATION "s3://sparsha-aws-cdl/";

In [0]:
%sql
DESCRIBE DATABASE sparsha_aws_cdl;

database_description_item,database_description_value
Namespace Name,sparsha_aws_cdl
Comment,
Location,s3://sparsha-aws-cdl/
Owner,root


In [0]:
%sql
USE sparsha_aws_cdl;

In [0]:
%sql
SHOW TABLES;

database,tableName,isTemporary


In [0]:
%sql
DROP TABLE IF EXISTS clicks_bronze;
DROP TABLE IF EXISTS clicks_silver;
DROP TABLE IF EXISTS clicks_with_profile;
DROP TABLE IF EXISTS clicks_with_profile_GOLD;
DROP TABLE IF EXISTS clicks_with_profile_GOLD_athena;

In [0]:
display(dbutils.fs.ls("/databricks-datasets"))

path,name,size,modificationTime
dbfs:/databricks-datasets/COVID/,COVID/,0,1655648398637
dbfs:/databricks-datasets/README.md,README.md,976,1532502324000
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0,1655648398637
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359,1455505834000
dbfs:/databricks-datasets/adult/,adult/,0,1655648398637
dbfs:/databricks-datasets/airlines/,airlines/,0,1655648398637
dbfs:/databricks-datasets/amazon/,amazon/,0,1655648398637
dbfs:/databricks-datasets/asa/,asa/,0,1655648398637
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0,1655648398637
dbfs:/databricks-datasets/bikeSharing/,bikeSharing/,0,1655648398637


In [0]:
%fs ls "s3://sparsha-aws-cdl"

In [0]:
#sparsha-aws-cdl S3 bucket working folders
dbutils.fs.rm("s3://sparsha-aws-cdl/clicks-bronze", True)
dbutils.fs.rm("s3://sparsha-aws-cdl/clicks-silver", True)
dbutils.fs.rm("s3://sparsha-aws-cdl/clicks-with-profile", True)
dbutils.fs.rm("s3://sparsha-aws-cdl/clicks_with_profile_GOLD", True)
dbutils.fs.rm("s3://sparsha-aws-cdl/checkpoint-clicks-bronze", True)
dbutils.fs.rm("s3://sparsha-aws-cdl/checkpoint-clicks-silver", True)

Out[3]: False

In [0]:
import pandas as pd

df = pd.read_csv("https://github.com/RecoHut-Datasets/ad-clicks/raw/main/ad-clicks.csv")
df = df.astype('str')
df.head()

Out[20]:

,timestamp,txId,userSessionId,teamId,userId,adId,adCategory
8641,2016-06-09 13:10:47,25973,20699,36,1953,0,sports
4371,2016-06-04 04:01:26,17524,16055,64,1567,14,fashion
12937,2016-06-13 15:17:22,33770,26385,2,1337,25,computers
14088,2016-06-14 13:33:43,36216,34479,57,2221,23,fashion
11539,2016-06-12 09:35:22,31432,26905,93,417,0,sports


In [0]:
df = spark.createDataFrame(df)
display(df.show(5))

+-------------------+-----+-------------+------+------+----+----------+
 timestamp| txId|userSessionId|teamId|userId|adId|adCategory|
+-------------------+-----+-------------+------+------+----+----------+
2016-06-09 13:10:47|25973| 20699| 36| 1953| 0| sports|
2016-06-04 04:01:26|17524| 16055| 64| 1567| 14| fashion|
2016-06-13 15:17:22|33770| 26385| 2| 1337| 25| computers|
2016-06-14 13:33:43|36216| 34479| 57| 2221| 23| fashion|
2016-06-12 09:35:22|31432| 26905| 93| 417| 0| sports|
+-------------------+-----+-------------+------+------+----+----------+
only showing top 5 rows

In [0]:
df.createOrReplaceTempView("clicks")
df.printSchema()

root
-- timestamp: string (nullable = true)
-- txId: string (nullable = true)
-- userSessionId: string (nullable = true)
-- teamId: string (nullable = true)
-- userId: string (nullable = true)
-- adId: string (nullable = true)
-- adCategory: string (nullable = true)

In [0]:
display(spark.sql("SELECT * FROM clicks LIMIT 10"))

timestamp,txId,userSessionId,teamId,userId,adId,adCategory
2016-06-09 13:10:47,25973,20699,36,1953,0,sports
2016-06-04 04:01:26,17524,16055,64,1567,14,fashion
2016-06-13 15:17:22,33770,26385,2,1337,25,computers
2016-06-14 13:33:43,36216,34479,57,2221,23,fashion
2016-06-12 09:35:22,31432,26905,93,417,0,sports
2016-05-28 16:50:52,8654,7541,11,1260,25,computers
2016-06-14 21:43:04,36906,34541,64,1157,21,movies
2016-06-10 16:40:49,28492,26681,61,1085,6,movies
2016-06-08 22:26:59,24980,20697,36,1001,12,computers
2016-06-08 06:07:48,23917,20977,90,1639,23,fashion


## Kinesis Stream Producer

<img src="https://drive.google.com/uc?id=1fZlcO7umZW5xtgcrkbGGSnKfX5IQeJ_E&authuser=recohut.data.001%40gmail.com&usp=drive_fs" height="600" width="800"/>

In [0]:
%scala
def get_kinesis_endpoint(): String = {
  "https://kinesis.us-east-1.amazonaws.com"
}

get_kinesis_endpoint: ()String

In [0]:
%scala
def get_partition_info(fullPath: String): Seq[com.databricks.backend.daemon.dbutils.FileInfo] = {
  // fullPath should be any DBFS mounted location 
  var input = fullPath
  // if fullPath ends with a "/" then we remove it to do some string adjustments 
  if ( fullPath.endsWith("/") ) { 
    input = fullPath.slice(0, fullPath.length - 1)
  }
  // Grab the last folder of the path 
  val endPath = input.split('/').last
  // Grab the prefix until the folder in question 
  val prefix = input.slice(0, input.length - endPath.length)
  // Filter and grab get the dbutils output for that given folder 
  val inputDir = dbutils.fs.ls(prefix).filter(x => x.name == endPath + "/")
  get_partition_helper(inputDir)
}

def get_partition_helper(paths: Seq[com.databricks.backend.daemon.dbutils.FileInfo]): Seq[com.databricks.backend.daemon.dbutils.FileInfo] = {
  val debug = false
  var ret = paths.filter(x => !x.name.startsWith("_"))
  var next = paths.flatMap(y => dbutils.fs.ls(y.path).filter(x => !x.name.startsWith("_")))
  if (debug) {
    ret.foreach(x => println(x.path))
  }
  if (next.filter(x => x.name contains ".parquet").length == 0) { 
    ret = get_partition_helper(paths.flatMap(y => dbutils.fs.ls(y.path).filter(x => !x.name.startsWith("_"))))
  } 
  ret 
}

get_partition_info: (fullPath: String)Seq[com.databricks.backend.daemon.dbutils.FileInfo]
get_partition_helper: (paths: Seq[com.databricks.backend.daemon.dbutils.FileInfo])Seq[com.databricks.backend.daemon.dbutils.FileInfo]

In [0]:
%scala
import com.amazonaws.services.kinesis.model.PutRecordRequest
import com.amazonaws.services.kinesis.AmazonKinesisClient
import com.amazonaws.auth.{DefaultAWSCredentialsProviderChain, BasicAWSCredentials}
import java.nio.ByteBuffer
import scala.util.Random
import com.google.gson.Gson
import org.joda.time
import org.joda.time.format._
import java.sql.Timestamp;
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

import com.amazonaws.services.kinesis.model.PutRecordRequest
import com.amazonaws.services.kinesis.AmazonKinesisClient
import com.amazonaws.auth.{DefaultAWSCredentialsProviderChain, BasicAWSCredentials}
import java.nio.ByteBuffer
import scala.util.Random
import com.google.gson.Gson
import org.joda.time
import org.joda.time.format._
import java.sql.Timestamp
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

In [0]:
%scala
// === Configurations for Kinesis streams ===

val kinesisStreamName = "sparsha-aws-cdl-clicks"
val kinesisEndpointUrl = get_kinesis_endpoint() // e.g. https://kinesis.us-east-1.amazonaws.com"

// === Configurations of amount of data to produce ===
val recordsPerSecond = 10
val wordsPerRecord = 10
val numSecondsToSend = 7200

kinesisStreamName: String = sparsha-aws-cdl-clicks
kinesisEndpointUrl: String = https://kinesis.us-east-1.amazonaws.com
recordsPerSecond: Int = 10
wordsPerRecord: Int = 10
numSecondsToSend: Int = 7200

In [0]:
%scala
// Create the low-level Kinesis Client from the AWS Java SDK.
// val kinesisClient = new AmazonKinesisClient(new BasicAWSCredentials(aws_kinesis_keys(0), aws_kinesis_keys(1)))
val kinesisClient = new AmazonKinesisClient() //using the IAM_Role

kinesisClient.setEndpoint(kinesisEndpointUrl)

println(s"Putting records onto stream $kinesisStreamName and endpoint $kinesisEndpointUrl at a rate of" +
  s" $recordsPerSecond records per second with $wordsPerRecord words per record for $numSecondsToSend seconds")

// Function to generate data

val impressions = spark.sql("select * from clicks").collect()

case class Record(timestamp: String, clickTimestamp: String, txId: String, userSessionId: String, teamId: String, userId: String, adId: String, adCategory: String)

def GsonTest() : String = {
    val imp = impressions(Random.nextInt(impressions.size)).toSeq
    val dt = new Timestamp(System.currentTimeMillis()-100)

    val r = Record(imp(0).asInstanceOf[String], dt.toString(), imp(1).asInstanceOf[String], imp(2).asInstanceOf[String], imp(3).asInstanceOf[String], imp(4).asInstanceOf[String], imp(5).asInstanceOf[String], imp(6).asInstanceOf[String])
    // create a JSON string from the Record, then print it
    val gson = new Gson
    val jsonString = gson.toJson(r)
    return jsonString
}

// Generate and send the data
for (round <- 1 to numSecondsToSend) {
  for (recordNum <- 1 to recordsPerSecond) {
    val data = GsonTest()
    println(data)
    val partitionKey = s"partitionKey-$recordNum"
    val putRecordRequest = new PutRecordRequest().withStreamName(kinesisStreamName)
        .withPartitionKey(partitionKey)
        .withData(ByteBuffer.wrap(data.getBytes()))
    kinesisClient.putRecord(putRecordRequest)
    println(putRecordRequest)
  }
  Thread.sleep(100) // Sleep for a second
  println(s"Sent $recordsPerSecond records with $wordsPerRecord words each")
}

println("\nTotal number of records sent")

Putting records onto stream sparsha-aws-cdl-clicks and endpoint https://kinesis.us-east-1.amazonaws.com at a rate of 10 records per second with 10 words per record for 7200 seconds
{"timestamp":"2016-06-15 18:38:52","clickTimestamp":"2022-06-19 16:17:17.422","txId":"38468","userSessionId":"33802","teamId":"156","userId":"994","adId":"0","adCategory":"sports"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=180 cap=180],PartitionKey: partitionKey-1,}
{"timestamp":"2016-06-09 13:10:47","clickTimestamp":"2022-06-19 16:17:17.475","txId":"25973","userSessionId":"20699","teamId":"36","userId":"1953","adId":"0","adCategory":"sports"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=180 cap=180],PartitionKey: partitionKey-2,}
{"timestamp":"2016-06-10 09:57:38","clickTimestamp":"2022-06-19 16:17:17.481","txId":"28096","userSessionId":"26402","teamId":"9","userId":"1652","adId":"1","adCategory":"sports"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=179 cap=179],PartitionKey: partitionKey-3,}
{"timestamp":"2016-06-06 16:48:01","clickTimestamp":"2022-06-19 16:17:17.487","txId":"21715","userSessionId":"20893","teamId":"75","userId":"1525","adId":"22","adCategory":"computers"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=184 cap=184],PartitionKey: partitionKey-4,}
{"timestamp":"2016-06-15 08:39:31","clickTimestamp":"2022-06-19 16:17:17.493","txId":"37643","userSessionId":"34557","teamId":"66","userId":"12","adId":"16","adCategory":"clothing"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=181 cap=181],PartitionKey: partitionKey-5,}
{"timestamp":"2016-06-07 13:51:39","clickTimestamp":"2022-06-19 16:17:17.499","txId":"22997","userSessionId":"22126","teamId":"133","userId":"1097","adId":"5","adCategory":"computers"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=184 cap=184],PartitionKey: partitionKey-6,}
{"timestamp":"2016-06-15 16:14:44","clickTimestamp":"2022-06-19 16:17:17.505","txId":"38225","userSessionId":"33440","teamId":"126","userId":"967","adId":"23","adCategory":"fashion"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=182 cap=182],PartitionKey: partitionKey-7,}
{"timestamp":"2016-05-29 15:36:46","clickTimestamp":"2022-06-19 16:17:17.512","txId":"9664","userSessionId":"7674","teamId":"53","userId":"670","adId":"21","adCategory":"movies"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=178 cap=178],PartitionKey: partitionKey-8,}
{"timestamp":"2016-06-14 11:20:28","clickTimestamp":"2022-06-19 16:17:17.518","txId":"36048","userSessionId":"34271","teamId":"20","userId":"1806","adId":"28","adCategory":"hardware"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=183 cap=183],PartitionKey: partitionKey-9,}
{"timestamp":"2016-06-15 23:11:11","clickTimestamp":"2022-06-19 16:17:17.524","txId":"38827","userSessionId":"34565","teamId":"66","userId":"426","adId":"17","adCategory":"games"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=179 cap=179],PartitionKey: partitionKey-10,}
Sent 10 records with 10 words each
{"timestamp":"2016-06-12 19:29:11","clickTimestamp":"2022-06-19 16:17:17.632","txId":"32272","userSessionId":"26897","teamId":"90","userId":"1972","adId":"2","adCategory":"electronics"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=185 cap=185],PartitionKey: partitionKey-1,}
{"timestamp":"2016-06-11 05:08:44","clickTimestamp":"2022-06-19 16:17:17.639","txId":"29260","userSessionId":"28618","teamId":"128","userId":"1090","adId":"18","adCategory":"games"}
{StreamName: sparsha-aws-cdl-clicks,Data: java.nio.HeapByteBuffer[pos=0 lim=181 cap=181],PartitionKey: partitionKey-2,}
{"timestamp":"2016-05-28 04:16:13","clickTimestamp":"2022-06-19 16:17:17.645","txId":"8105","userSessionId":"7795","teamId":"82","userId":"1214","adId":"0"